In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

In [2]:
df = pd.read_csv("../data/diamonds.csv", index_col='diamond_id')

In [3]:
df.head(2)

,Unnamed: 0,shape,size,color,fancy_color_dominant_color,fancy_color_secondary_color,fancy_color_overtone,fancy_color_intensity,clarity,cut,...,girdle_min,girdle_max,culet_size,culet_condition,fluor_color,fluor_intensity,lab,total_sales_price,eye_clean,date
diamond_id,,,,,,,,,,,,,,,,,,,,,
131328926,0,Round,0.09,E,NaN,NaN,NaN,NaN,VVS2,Excellent,...,M,M,N,NaN,NaN,None,IGI,200,NaN,2022-02-24
131704776,1,Round,0.09,E,NaN,NaN,NaN,NaN,VVS2,Very Good,...,STK,STK,N,NaN,NaN,None,IGI,200,NaN,2022-02-24


# EDA

---

 1. First, for the categoricial variables, how much variety is there?
 2. Second, what can reasonably be ditched?
 3. Lastly, for the numerical data, what are ranges?

In [4]:
df.describe()

,Unnamed: 0,size,depth_percent,table_percent,meas_length,meas_width,meas_depth,total_sales_price
count,219704.000000,219704.000000,219704.000000,219704.000000,219704.000000,219704.000000,219704.000000,2.197040e+05
mean,109851.500000,0.755176,61.683794,57.747579,5.548867,5.135623,3.285698,6.908035e+03
std,63423.226108,0.845892,9.915251,9.959906,1.763932,1.374527,2.054817,2.595944e+04
min,0.000000,0.080000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000e+02
25%,54925.750000,0.310000,61.200000,57.000000,4.350000,4.310000,2.680000,9.580000e+02
50%,109851.500000,0.500000,62.400000,58.000000,5.060000,4.800000,3.030000,1.970000e+03
75%,164777.250000,1.000000,63.500000,60.000000,6.350000,5.700000,3.630000,5.207000e+03
max,219703.000000,19.350000,98.700000,94.000000,93.660000,62.300000,76.300000,1.449881e+06


In [18]:
df['date'].unique() # clearly this can go as can the ID

array(['2022-02-24'], dtype=object)

In [33]:
for col in df:
    print(f"There are {df[col].isnull().sum()} nans in {col}")

There are 0 nans in Unnamed: 0
There are 0 nans in shape
There are 0 nans in size
There are 9162 nans in color
There are 210540 nans in fancy_color_dominant_color
There are 218642 nans in fancy_color_secondary_color
There are 217666 nans in fancy_color_overtone
There are 210542 nans in fancy_color_intensity
There are 0 nans in clarity
There are 60607 nans in cut
There are 0 nans in symmetry
There are 0 nans in polish
There are 0 nans in depth_percent
There are 0 nans in table_percent
There are 0 nans in meas_length
There are 0 nans in meas_width
There are 0 nans in meas_depth
There are 83433 nans in girdle_min
There are 84296 nans in girdle_max
There are 85741 nans in culet_size
There are 204385 nans in culet_condition
There are 203978 nans in fluor_color
There are 128 nans in fluor_intensity
There are 0 nans in lab
There are 0 nans in total_sales_price
There are 156917 nans in eye_clean
There are 0 nans in date


### Ok, let's get this cleaned up. 

In [34]:
df.drop(['Unnamed: 0', 'date'], axis=1, inplace=True)

### Here's my hypothesis: Size is the main driver.

I don't think people really care about color because they really can't see it easily. Cuts I think are a matter of perference. A bad cut condition might have some negative impact but I don't think it will be much. Also, I'm  putting my money on a boosted model like `XGBoost`.